AutoGIS Final Assignment, Hanna Hirvonen

AccessViz

The Helsinki Region Travel Time Matrix Data can be downloaded from http://blogs.helsinki.fi/accessibility/helsinki-region-travel-time-matrix/

FileFinder

In [1]:
# Importing the necessary modules:
import geopandas as gpd
import glob

# Creating an empty list for the YKR IDs and the filepaths:
ykr_ids = []
user_fps = []

# The user creates a list of YKR IDs:
n = int(input('Enter the number of YKR grid cells to be analysed:'))
print('Enter the YKR ids one by one')
for i in range(0, n):
    ele = int(input())
    ykr_ids.append(ele)

Enter the number of YKR grid cells to be analysed: 4


Enter the YKR ids one by one


 5785640
 5785641
 5785642
 5785643


In [2]:
# The user defines the input folder:
input_fp = str(input('Enter the filepath to the input folder:'))

Enter the filepath to the input folder: data/


In [3]:
# Searching for corresponding .txt files from the input folder:
num = 1
for i in ykr_ids:
    if num != len(ykr_ids):
        print('Process: ', num, '/', len(ykr_ids), ' -  Processing the file:', file)
    else:
        print('Process: ', num, '/', len(ykr_ids), ' -  Processing the file:', i)
        print('All done.')
    file = input_fp + 'travel_times_to_ ' + str(i) + '.txt'
    user_fps.append(glob.glob(file))
    num = num + 1

Process:  1 / 4  -  Processing the file: 5785640
Process:  2 / 4  -  Processing the file: 5785641
Process:  3 / 4  -  Processing the file: 5785642
Process:  4 / 4  -  Processing the file: 5785643
All done.


In [4]:
user_fps

[['data/travel_times_to_ 5785640.txt'],
 ['data/travel_times_to_ 5785641.txt'],
 ['data/travel_times_to_ 5785642.txt'],
 ['data/travel_times_to_ 5785643.txt']]